#Importing the libraries

In [ ]:
import rdkit as rd
from rdkit import Chem
from rdkit.Chem import rdDistGeom
from rdkit.Chem import AllChem

#Importing Molecular List as SMILES

In [ ]:
mList = ['CCC1CC(CCC1=O)C(=O)C1=CC=CC(C)=C1','CCC1CC(CCC1=O)C(=O)C1=CC=CC=C1',\
         'CC(=C)C(C1=CC=CC=C1)S(=O)CC(N)=O','CC1=CC(=CC=C1)C(C1CCC(N)CC1)C(F)(F)F',\
         'CNC1CCC(C2=CC(Cl)=C(Cl)C=C2)C2=CC=CC=C12','CCCOC(C1CCCCC1)C1=CC=C(Cl)C=C1'] 

#Converting SMILES to mols

In [ ]:
mMols = [Chem.MolFromSmiles(m) for m in mList]

In [ ]:
mMols

#Adding/displaying the Hydrogens to the molecules for getting realistic geometries considering all the hydrogens present in the molecule.

In [ ]:
mMols_AddHs = [Chem.AddHs(mMol) for mMol in mMols]

#ETKDG is the default conformer generation method after distance geometry has been applied to generate the conformers. ETKDGs uses the torsion angle preferences to correct the conformers.

In [ ]:
param = rdDistGeom.ETKDGv2()

#Setting the number of conformers that should be generated

In [ ]:
numconf = 1

#Generation of multiple conformers using embedding method

In [ ]:
cids = [rdDistGeom.EmbedMultipleConfs(mol,numconf,rdDistGeom.ETKDGv2()) for mol in mMols_AddHs]

In [ ]:
cids

#Energy Minimization of the generated conformers using MMFF 

In [ ]:
mps = [AllChem.MMFFGetMoleculeProperties(mol, mmffVariant = "MMFF94s") for mol in mMols_AddHs]

In [ ]:
mps

#Optimization of the molecular conformers

In [ ]:
[AllChem.MMFFOptimizeMoleculeConfs(mol, mmffVariant = "MMFF94s") for mol in mMols_AddHs]

In [ ]:
[AllChem.MMFFOptimizeMoleculeConfs(mMols_AddHs[1], mps[1], confID = cids[1])]

In [ ]:
for (x, y, z) in zip(mMols_AddHs , mps,cids):
    [AllChem.MMFFOptimizeMoleculeConfs(x, y, confID = z)]

In [ ]:
mMols_AddHs

In [ ]:
mps

In [ ]:
cids

In [ ]:
Chem.AllChem.GenerateDepictionMatching3DStructure()


In [ ]:
import py3Dmol
view = py3Dmol.view(query='pdb:1hvr')
view.setStyle({'cartoon':{'color':'spectrum'}})
view

#Display 3D molecule from the forumula (smiles) using rdkit and py3Dmol - Shared by Sharmeen

In [ ]:
import py3Dmol

In [ ]:
def show(smi, style = "stick"):
    #Construct a molecule from a SMILES string.
    mol = Chem.MolFromSmiles(smi)
    #Adds hydrogens to the graph of a molecule.
    mol = Chem.AddHs(mol)
    #Use distance geometry to obtain initial coordinates for a molecule
    AllChem.EmbedMolecule(mol)
    #uses MMFF to optimize a molecule's structure
    AllChem.MMFFOptimizeMolecule(mol,maxIters = 200,mmffVariant = "MMFF94s")
    mblock = Chem.MolToMolBlock(mol)
    view = py3Dmol.view(width = 200, height = 200)
    view.addModel(mblock,"mol")
    view.setStyle({style:{}})
    view.zoomTo()
    view.show()

In [ ]:
show("CCCC1=NN(C2=C1N=C(NC2=O)C3=C(C=CC(=C3)S(=O)(=O)N4CCN(CC4)C)OCC)C")

In [ ]:
def show(smi, style = "stick"):
    #Construct a molecule from a SMILES string.
    mol = Chem.MolFromSmiles(smi)
    #Adds hydrogens to the graph of a molecule.
    mol = Chem.AddHs(mol)
    #Use distance geometry to obtain initial coordinates for a molecule
    AllChem.EmbedMolecule(mol)
    param = rdDistGeom.ETKDGv2()
    cids = rdDistGeom.EmbedMultipleConfs(mol,10,rdDistGeom.ETKDGv2())
    mps = [AllChem.MMFFGetMoleculeProperties(cid,mmffVariant= "MMFF94s") for cid in cids]
    [AllChem.MMFFOptimizeMolecule(mp,maxIters = 200,mmffVariant = "MMFF94s") for mp in mps]
    #uses MMFF to optimize a molecule's structure
    #AllChem.MMFFOptimizeMolecule(mol,maxIters = 200,mmffVariant = "MMFF94s")
    mblock = Chem.MolToMolBlock(mol)
    view = py3Dmol.view(width = 200, height = 200)
    view.addModel(mblock,"mol")
    view.setStyle({style:{}})
    view.zoomTo()
    view.show()

In [ ]:
m = Chem.MolFromSmiles("C1CCC1OC")
m2 = Chem.AddHs(m)
#Use distance geometry to obtain multiple sets of coordinates for a molecule
cids = AllChem.EmbedMultipleConfs(m2,20,pruneRmsThresh = 1)
#AllChem.MMFFGetMoleculeProperties(m2,mmffVariant = "MMF94s")

In [ ]:
for cid in cids: AllChem.MMFFOptimizeMolecule(m2,confId = cid)

In [ ]:
view = py3Dmol.view(width = 200, height = 200)
view.addModel(m2,confId = cid,"mol")
view.setStyle({style:{}})
view.zoomTo()
view.show()

In [ ]:
m2

In [ ]:
rmslist = []
AllChem.AlignMolConformers(m2,RMSlist = rmslist)
print(len(rmslist))

rmslist

In [ ]:
rmslist

In [ ]:
res = AllChem.MMFFOptimizeMoleculeConfs(m2)